## **Sequential chains**

Some problems can only be solved sequentially. 

Consider a chatbot used to create a travel itinerary

We need to tell the chatbot our destination, receive suggestions on what to see on our trip, and tell the model which activities to select to compile the itinerary.
<div style="text-align: center;">
    <img src='./images/sequential-problem.png' width=50%>
</div>

This is a sequential problem, as it requires more than one user input:
- One to specify the destination
- One to select the activities

In sequential chains, the output from one chain becomes the input to another.

We'll create two prompt templates: one to generate suggestions for activities from the input destination, and another to create an itinerary for one day of activities from the model's top three suggestions. 

```python
destination_prompt = PromptTemplate(
    input_variables = ["destination"],
    template = "I am planning a trip to {destination}. Can you suggest some activities to do there?"
)
activities_prompt = PromptTemplate(
    input_variables = ["activities"],
    template = "I only have one day, so can you create an itinerary from your top three activities: {activities}"
)
```

We define our model, and begin our sequential chain. 

```python
llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key="<OPENAI_API_KEY>"
)
```

We start by defining a dictionary that passes our destination prompt template to the LLM and parses the output to a string, all using LCEL's pipe. This gets assigned to the "activities" key, which is important, as this is the input variable to the second prompt template. We pipe the first chain into the second prompt template, then into the LLM, and again, parse to a string. We also wrap the sequential chain in parentheses so we can split this code across multiple lines. 

```python
seq_chain = (
    {"activities": destination_prompt | llm | StrOutputParser()}
    | activities_prompt
    | llm
    | StrOutputParser()
)
```

To summarize: the `destination_prompt` is passed to the `LLM` to generate the activity suggestions, and the output is parsed to a string and assigned to `"activities"`. This is passed to the second `activities_prompt`, which is passed to the `LLM` to generate the itinerary, which is parsed as a string.

`"activities"` (`destination_prompt` → `llm` (this generates activity suggestions) → parse the suggestions output to string by using `StrOutputParser()`) →  pass `activities` to `activities_prompt` → pass `activities_prompt` to `llm` (this generates the itinerary) → parse itinerary output to string by using `StrOutputParser()`.

Let's invoke the hain, passing "Rome" as our input destination.

```python
print(seq_chain.invoke({"destination": "Rome"}))
```

<img src='./images/sequential-travel.png' width=70%>

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser   # to parse the output to string
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

destination_prompt = PromptTemplate(
    input_variables = ["destination"],
    template = "I am planning a trip to {destination}. Can you suggest some activities to do there?"
)
activities_prompt = PromptTemplate(
    input_variables = ["activities"],
    template = "I only have one day, so can you create an itinerary from your top three activities: {activities}"
)

llm = ChatOpenAI(
    model="gpt-4o-mini",
    api_key=api_key
)

seq_chain = (
    {"activities": destination_prompt | llm | StrOutputParser()}
    | activities_prompt
    | llm
    | StrOutputParser()
)

print(seq_chain.invoke({"destination": "Rome"}))

Certainly! Here’s a one-day itinerary focusing on three top activities to make the most of your time in Rome:

### Morning

**1. Visit the Colosseum** (around 9:00 AM)  
- Start your day early at the Colosseum, one of the city’s most iconic landmarks. Book a timed entry ticket in advance to skip the lines and spend some time exploring the grandeur of this ancient amphitheater. Don’t miss the underground section and the upper tiers for incredible views.

**2. Explore the Roman Forum** (around 11:00 AM)  
- Just a short walk from the Colosseum, the Roman Forum is a captivating site filled with ruins that tell the story of Ancient Rome. Spend about an hour here, taking in the historical significance and imagining what life was like during the empire's height.

### Lunch

**Lunch in Trastevere** (around 12:30 PM)  
- Head to the charming neighborhood of Trastevere for lunch. Choose a local trattoria to enjoy authentic Roman cuisine—try dishes like cacio e pepe or pasta alla carbonara. Don’

## Example

In [2]:
# Create a prompt template that takes an input activity
learning_prompt = PromptTemplate(
    input_variables=["activity"],
    template="I want to learn how to {activity}. Can you suggest how I can learn this step-by-step?"
)

# Create a prompt template that places a time constraint on the output
time_prompt = PromptTemplate(
    input_variables= ["learning_plan"],
    template="I only have one week. Can you create a plan to help me hit this goal: {learning_plan}."
)

# Invoke the learning_prompt with an activity
print(learning_prompt.invoke({"activity": "play golf"}))

text='I want to learn how to play golf. Can you suggest how I can learn this step-by-step?'


In [3]:
learning_prompt = PromptTemplate(
    input_variables=["activity"],
    template="I want to learn how to {activity}. Can you suggest how I can learn this step-by-step?"
)

time_prompt = PromptTemplate(
    input_variables=["learning_plan"],
    template="I only have one week. Can you create a plan to help me hit this goal: {learning_plan}."
)

# Complete the sequential chain with LCEL
seq_chain = ({"learning_plan": learning_prompt | llm | StrOutputParser()}
    | time_prompt
    | llm
    | StrOutputParser())

# Call the chain
print(seq_chain.invoke({"activity": "play classic guitar"}))

Certainly! Since you have a week to kickstart your classical guitar journey, here's a focused 7-day plan that incorporates the steps you've outlined while ensuring you stay engaged and make solid progress.

### 7-Day Classical Guitar Learning Plan

#### **Day 1: Set-Up and Familiarization**
- **Equipment Setup**: Get your classical guitar, tuner, metronome, and footstool ready.
- **Learn the Parts of the Guitar**: Spend 15-30 minutes understanding the anatomy of the guitar.
- **Posture Practice**: Sit with the guitar and practice holding it correctly for 10-15 minutes.
- **Tuning**: Use a tuner to learn how to tune your guitar properly.
  
#### **Day 2: Music Theory Basics**
- **Notes and Frets**: Spend 30 minutes learning the names of the strings (E, A, D, G, B, E) and the first 5 frets.
- **Reading Music**: Dedicate 30 minutes to understanding basic music notation and tablature through online resources or beginner books.
- **Finger Placement**: Work on placing your left-hand fingers 

## **Introduction to LangChain agents**

### **What are agents?**


  <div style="display: flex; align-items: flex-start;">
    <!-- Left Column -->
    <div style="width: 30%; padding: 10px;">
    In LangChain, agents use language models to determine actions. <br><br>
    Agents often use _tools_, which are functions called by the agent to interact with the system. These tools can be high-level utilities to transform inputs, or they can be task-specific. <br><br>
    Agents can even use chains and other agents as tools! In this course, we'll discuss a type of agent called _ReAct agents_.
    </div>
    <!-- Right Column -->
    <div style="width: 33%; padding: 10px;">
    <div>
    <img src='./images/agent.png' width=90%>
    </div>
    </div>
</div>

### **ReAct agents**

__ReAct__ stands for **Reason** and **Act**. And this is exactly how the agent operates.

It prompts the model using a repeated loop of thinking, acting, and observing. If we were to ask a ReAct agent that had access to a weather tool, "What is the weather like in Kingston, Jamaica?", it would start by thinking about the task and which tool to call, call that tool using the information, and observe the results from the tool call.

<img src='./images/react-loop.png' width=35%>

To implement agents, we'll be using __LangGraph__, which is branch of the LangChain ecosystem specifically _for designing agentic systems_, or _systems including agents_. 

Like LangChain's core library, it's is built to provide a unified, tool-agnostic syntax. We'll be using the version 0.066 of LangGraph in this course.

### **ReAct agent**

We'll create a ReAct agent that can solve math problems - something most LLMs struggle with. 

1. We import `create_react_agent` from `langgraph` and the `load_tools()` function. 
2. We initialize our LLM, and load the `llm-math` tool using the `load_tools()` function. 
3. To create the agent, we pass the LLM and tools to `create_react_agent()`.
4. Just like chains, agents can be executed with the `.invoke()` method. 
5. We pass the chat model a message to find the square root of `101`, which isn't a whole number.

Let's see how the agent approaches the problem!

```python
from langgraph.prebuilt import create_react_agent                                         # 1.
from langchain_community.agent_toolkits.load_tools import load_tools                      # 1.

llm = ChatOpenAI(model="gpt-4o-mini", api_key="<OPENAI_API_KEY>")                         # 2.    
tools = load_tools(["llm-math"], llm=llm)                                                 # 2.
agent = create_react_agent(llm, tools)                                                    # 3.

messages = agent.invoke({"messages":[("human", "What is the square root of 101?")]})      # 4.
print(messages)
```

In [8]:
from langgraph.prebuilt import create_react_agent
from langchain_community.agent_toolkits.load_tools import load_tools

llm = ChatOpenAI(model="gpt-4o-mini", api_key=api_key) 
tools = load_tools(["llm-math"], llm=llm)
agent = create_react_agent(llm, tools)

messages = agent.invoke({"messages":[("human", "What is the square root of 101?")]})
print(messages)

{'messages': [HumanMessage(content='What is the square root of 101?', additional_kwargs={}, response_metadata={}, id='fec1a470-f5a9-462a-82d4-0e7a5fdca362'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_3mzOcQ0AByTlZTKl6AEtP9po', 'function': {'arguments': '{"__arg1":"sqrt(101)"}', 'name': 'Calculator'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 63, 'total_tokens': 83, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_709714d124', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-a78bfd2d-29bf-4e88-b8f0-64814bd9ea9d-0', tool_calls=[{'name': 'Calculator', 'args': {'__arg1': 'sqrt(101)'}, 'id': 'call_3mzOcQ0AByTlZTKl6AEtP9po', 'type': 'tool_call'}], usage_metadata={

There's a lot of metadata in the output:
```python
{'messages': [
HumanMessage(content='What is the square root of 101?', ...)

AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"__arg1":"sqrt(101)"}', 'name': 'Calculator'}}]}, ...)

ToolMessage(content='Answer: 10.04987562112089', name='Calculator', ...)

AIMessage(content='The square root of 101 is approximately 10.05.', ...)
]}
```

The first is our prompt defining the problem.

The second is created by the model to identify the tool to use and to convert our query into mathematical format.

The third is the result of the tool call.

The final message is the model's response after ovserving the tool's answer, which it decided to roun to two decimal places.

If we just want the final response, we can subset the final message and extract it's content with the `.content` attribute.

In [9]:
print(messages['messages'][-1].content)

The square root of 101 is approximately 10.05.


## Example

In [10]:
import wikipedia

# Define the tools
tools = load_tools(["wikipedia"])

# Define the agent
agent = create_react_agent(llm, tools)

# Invoke the agent
response = agent.invoke({"messages": [("human", "How many people live in New York City?")]})
print(response['messages'][-1].content)

As of 2023, the estimated population of New York City is 8,258,035.


## **Custom tools for agents**

Tools in LangChain must be formatted in a specific way to be compatible with agents. They must have a name, accessible via the `.name` attribute. A description, which is used by the LLM to determine when to call the tool.

In [14]:
print("Name of the tool: " + tools[0].name + "\n")
print("Description of the tool " + tools[0].name + ": " + tools[0].description)

Name of the tool: wikipedia

Description of the tool wikipedia: A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.


Finally, the return_direct parameter indicates whether the agent should stop after invoking this tool, which it won't in this case. Understanding this required format will help us to understand how to create our own tools.

In [15]:
print(tools[0].return_direct)

False


## **Defining a custom function**

Let's say we want to define a Python function to generate a financial report for a company. It takes three arguments: 
- The `company_name`,
- `revenue`,
- and `expenses`.

And outputs - a string containing the `net_income`. We make the use of this function clear in the _docstring_, defined using triple quotes.

In [25]:
def financial_report(company_name: str, revenue: int, expenses: int) -> str:
    """Generate a financial report for a company that calculates net income"""
    net_income = revenue - expenses

    report = f"Financial Report for {company_name}:\n"
    report += f"Revenue: ${revenue}\n"
    report += f"Expenses: ${expenses}\n"
    report += f"Net Income: ${net_income}\n"
    return report

Here's what the report looks like:

In [17]:
print(financial_report(company_name="LemonadeStand", revenue=100, expenses=50))

Financial Report for LemonadeStand:
Revenue: $100
Expenses: $50
Net Income: $50



Let's convert this function into a tool our agent can call.

### **From functions to tools**

To do this, we import the `@tool` __decorator__ and add it before the function definition. The @tool modifies the function so it's in the correct format to be used by a tool.

In [19]:
from langchain_core.tools import tool

@tool               # <--- This is the new line
def financial_report(company_name: str, revenue: int, expenses: int) -> str:
    """Generate a financial report for a company that calculates net income"""
    net_income = revenue - expenses

    report = f"Financial Report for {company_name}:\n"
    report += f"Revenue: ${revenue}\n"
    report += f"Expenses: ${expenses}\n"
    report += f"Net Income: ${net_income}\n"
    return report

### **Examining the new tool**

Like with the built-in tool we were looking at, we can now examine the various attributes of our tool. These include its name, which is the function name by default, its description, which is the function's docstring, and `return_direct`, which is set to False by default. We can also print the tools arguments, which lay out the argument names and expected data types.

In [21]:
print(financial_report.name)
print(financial_report.description)
print(financial_report.return_direct)
print(financial_report.args)

financial_report
Generate a financial report for a company that calculates net income
False
{'company_name': {'title': 'Company Name', 'type': 'string'}, 'revenue': {'title': 'Revenue', 'type': 'integer'}, 'expenses': {'title': 'Expenses', 'type': 'integer'}}


## **Integrating the custom tool**

We'll again use a ReAct agent, combining the chat LLM with a list of tools to use, containing our new custom tool. We invoke the agent with an input containing the required information: a _company name_, _revenue_, and _expenses_.

In [22]:
from langgraph.prebuilt import create_react_agent

llm = ChatOpenAI(
    model = "gpt-4o-mini",
    api_key =api_key
)
agent = create_react_agent(llm, [financial_report])

messages = agent.invoke({"messages": [("human", "TechStack generated made $10 million with $5 million of costs. Generate a financial report.")]})
print(messages)

{'messages': [HumanMessage(content='TechStack generated made $10 million with $5 million of costs. Generate a financial report.', additional_kwargs={}, response_metadata={}, id='03aff138-cc90-4329-bf0b-d741d11013c0'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_JB0rEfU9Qj2G43ZpLAMS34SE', 'function': {'arguments': '{"company_name":"TechStack","revenue":10000000,"expenses":5000000}', 'name': 'financial_report'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 77, 'total_tokens': 108, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_709714d124', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6f406c05-0991-450f-bc0d-de234ed3feba-0', tool_calls=[{'name': 'financial_rep

The response from the agent starts with our input, then determines that the financial_report tool should be called, which returns a tool message containing the output from our function, and finally, the output is passed to the LLM, which responds to us. Let's zoom in on this final message.

In [28]:
print(messages['messages'][-1].content)

Here's the financial report for TechStack:

- **Revenue:** $10,000,000
- **Expenses:** $5,000,000
- **Net Income:** $5,000,000 

The company has achieved a net income of $5 million.


In [30]:
# Recal the financial_report tool directly
print(financial_report(company_name="TechStack", revenue=10000000, expenses=5000000))

Financial Report for TechStack:
Revenue: $10000000
Expenses: $5000000
Net Income: $5000000



Notice that ther's slight formatting differences between the two: the LLM received the tool output, and put it's own slight spin on it, which we may need to watch out for.

## Example

In [31]:
import pandas as pd

df = pd.read_csv("./datasets/customers.csv")

customers = pd.DataFrame(df)

# Define a function to retrieve customer info by-name
def retrieve_customer_info(name: str) -> str:
    """Retrieve customer information based on their name."""
    # Filter customers for the customer's name
    customer_info = customers[customers['name'] == name]
    return customer_info.to_string()
  
# Call the function on Peak Performance Co.
print(retrieve_customer_info("Peak Performance Co."))

    id                  name subscription_type  active_users  auto_renewal
3  104  Peak Performance Co.           Premium           800          True


In [32]:
# Convert the retrieve_customer_info function into a tool
@tool
def retrieve_customer_info(name: str) -> str:
    """Retrieve customer information based on their name."""
    customer_info = customers[customers['name'] == name]
    return customer_info.to_string()
  
# Print the tool's arguments
print(retrieve_customer_info.args)

{'name': {'title': 'Name', 'type': 'string'}}


In [37]:
# Create a ReAct agent
agent = create_react_agent(llm, [retrieve_customer_info])

# llm has already beed defined in the few cells above
llm = ChatOpenAI(
    model = "gpt-4o-mini",
    api_key =api_key
)

# Invoke the agent on the input
messages = agent.invoke({"messages": [("human", "Create a summary and a few sentences of the stats of our customer: Peak Performance Co.")]})
print(messages['messages'][-1].content)

**Summary of Customer: Peak Performance Co.**

- **ID:** 104
- **Subscription Type:** Premium
- **Active Users:** 800
- **Auto-Renewal:** Enabled

**Statistics Overview:**
Peak Performance Co. holds a Premium subscription with an active user base of 800. The account is set to auto-renew, ensuring uninterrupted service for their team.
